In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import scale

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
label = train_df.target
train = train_df.drop(['ID_code','target'],axis=1)

In [4]:
test = pd.read_csv('test.csv')

In [5]:
test_filtered = pd.read_pickle('test_filtred.pkl')

In [6]:
test_filtered = test_filtered.loc[:,train.columns]

In [7]:
test = test.drop(['ID_code'],axis=1)

In [8]:
train_test = pd.concat([train,test_filtered]).reset_index(drop=True)

In [9]:
vcs_train = {}
vcs_test = {}
vcs_train_test = {}


for col in tqdm(train.columns):
#     vcs_train[col] = train.loc[:,col].value_counts()/200000
#     vcs_test[col] = test_filtered.loc[:,col].value_counts()/100000
    vcs_train_test[col] = train_test.loc[:,col].value_counts()/300000

In [10]:
for col in tqdm(train.columns):
#     vtrain = vcs_train[col]
#     vtest = vcs_test[col]
    vtraintest = vcs_train_test[col]
    
#     train[col+'_train_vcs'] = vtrain[train[col]].fillna(0).values
#     train[col+'_test_vcs'] = vtest[train[col]].fillna(0).values
    t = vtraintest[train[col]].fillna(0).values
    train[col+'_train_test_sum_vcs'] = t
    
    train[col+'_train_test_sum_vcs_product'] = train[col]*t
    train[col+'_train_test_sum_vcs_div'] = train[col]/t
    
    train[col+'_train_test_sum_vcs_min'] = scale(train[col]) - scale(t)
    train[col+'_train_test_sum_vcs_plus'] = scale(train[col]) + scale(t)


    
#     test[col+'_train_vcs'] = vtrain[test[col]].fillna(0).values
#     test[col+'_test_vcs'] = vtest[test[col]].fillna(0).values
    t = vtraintest[test[col]].fillna(0).values
    test[col+'_train_test_sum_vcs'] = t
    
    test[col+'_train_test_sum_vcs_product'] = test[col]*t
    test[col+'_train_test_sum_vcs_div'] = test[col]/t  
    
    test[col+'_train_test_sum_vcs_min'] = scale(test[col]) - scale(t)     
    test[col+'_train_test_sum_vcs_plus'] = scale(test[col]) + scale(t)   

In [11]:
import numpy as np
import catboost as cb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
import warnings
from bayes_opt.observer import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs


In [16]:
params = {
    'task_type': 'GPU',
    'iterations': 20000,
    'loss_function': 'Logloss',
    'eval_metric':'AUC',
    'random_seed': 4242,
    "learning_rate": 0.03,
    "l2_leaf_reg": 3.0,
    'bagging_temperature': 1,
    'random_strength': 1,
    'depth': 4,
    'border_count':128}

folds = StratifiedKFold(n_splits=5, shuffle=False, random_state=99999)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
gains = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, label.values)):
    print("Fold {}".format(fold_))
    trn_data = cb.Pool(train.iloc[trn_idx], label=label.iloc[trn_idx])
    val_data = cb.Pool(train.iloc[val_idx], label=label.iloc[val_idx])
    clf = cb.CatBoostClassifier(**params)
    clf.fit(trn_data, eval_set = val_data,use_best_model = True,verbose=500,early_stopping_rounds=300)
    oof[val_idx] = clf.predict_proba(train.iloc[val_idx])[:,1]
    predictions += clf.predict_proba(test)[:,1] / folds.n_splits
    gains.append(clf.get_feature_importance(trn_data))

print("CV score: {:<8.5f}".format(roc_auc_score(label, oof)))

Fold 0


0:	learn: 0.5167695	test: 0.5116116	best: 0.5116116 (0)	total: 23.7ms	remaining: 7m 54s
500:	learn: 0.8701462	test: 0.8572521	best: 0.8572521 (500)	total: 7.73s	remaining: 5m
1000:	learn: 0.8977396	test: 0.8824428	best: 0.8824428 (1000)	total: 15.2s	remaining: 4m 49s
1500:	learn: 0.9108679	test: 0.8931750	best: 0.8931750 (1500)	total: 22.8s	remaining: 4m 40s
2000:	learn: 0.9191165	test: 0.8991625	best: 0.8991625 (2000)	total: 30.1s	remaining: 4m 31s
2500:	learn: 0.9251368	test: 0.9029232	best: 0.9029232 (2500)	total: 37.7s	remaining: 4m 23s
3000:	learn: 0.9299221	test: 0.9052183	best: 0.9052183 (3000)	total: 45.3s	remaining: 4m 16s
3500:	learn: 0.9339080	test: 0.9068377	best: 0.9068425 (3498)	total: 52.7s	remaining: 4m 8s
4000:	learn: 0.9374025	test: 0.9080669	best: 0.9080687 (3998)	total: 60s	remaining: 3m 59s
4500:	learn: 0.9405581	test: 0.9092057	best: 0.9092057 (4500)	total: 1m 7s	remaining: 3m 52s
5000:	learn: 0.9434482	test: 0.9101657	best: 0.9101665 (4998)	total: 1m 14s	remainin

0:	learn: 0.5173729	test: 0.5090862	best: 0.5090862 (0)	total: 20.2ms	remaining: 6m 44s
500:	learn: 0.8698494	test: 0.8585766	best: 0.8585766 (500)	total: 7.43s	remaining: 4m 49s
1000:	learn: 0.8978320	test: 0.8836948	best: 0.8836948 (1000)	total: 14.9s	remaining: 4m 43s
1500:	learn: 0.9106639	test: 0.8938592	best: 0.8938592 (1500)	total: 22.3s	remaining: 4m 35s
2000:	learn: 0.9189153	test: 0.8993252	best: 0.8993252 (2000)	total: 29.9s	remaining: 4m 29s
2500:	learn: 0.9249654	test: 0.9026211	best: 0.9026211 (2500)	total: 37.4s	remaining: 4m 21s
3000:	learn: 0.9298058	test: 0.9049479	best: 0.9049479 (3000)	total: 44.9s	remaining: 4m 14s
3500:	learn: 0.9339119	test: 0.9066552	best: 0.9066552 (3500)	total: 52.2s	remaining: 4m 6s
4000:	learn: 0.9374010	test: 0.9076561	best: 0.9076751 (3984)	total: 59.7s	remaining: 3m 58s
4500:	learn: 0.9406741	test: 0.9085589	best: 0.9085589 (4500)	total: 1m 7s	remaining: 3m 51s
5000:	learn: 0.9436417	test: 0.9092663	best: 0.9092663 (5000)	total: 1m 14s	re

0:	learn: 0.5193516	test: 0.5097912	best: 0.5097912 (0)	total: 23.7ms	remaining: 7m 54s
500:	learn: 0.8680785	test: 0.8594143	best: 0.8594143 (500)	total: 7.52s	remaining: 4m 52s
1000:	learn: 0.8969240	test: 0.8866158	best: 0.8866158 (1000)	total: 15s	remaining: 4m 44s
1500:	learn: 0.9099327	test: 0.8977337	best: 0.8977337 (1500)	total: 22.4s	remaining: 4m 35s
2000:	learn: 0.9181633	test: 0.9030774	best: 0.9030774 (2000)	total: 29.9s	remaining: 4m 28s
2500:	learn: 0.9241639	test: 0.9063798	best: 0.9063798 (2500)	total: 37.5s	remaining: 4m 22s
3000:	learn: 0.9290924	test: 0.9085702	best: 0.9085702 (3000)	total: 44.9s	remaining: 4m 14s
3500:	learn: 0.9332643	test: 0.9099440	best: 0.9099440 (3500)	total: 52.4s	remaining: 4m 7s
4000:	learn: 0.9368693	test: 0.9110681	best: 0.9110754 (3999)	total: 60s	remaining: 3m 59s
4500:	learn: 0.9401413	test: 0.9120812	best: 0.9120812 (4500)	total: 1m 7s	remaining: 3m 51s
5000:	learn: 0.9431501	test: 0.9127777	best: 0.9127777 (5000)	total: 1m 14s	remain

0:	learn: 0.5181547	test: 0.5207097	best: 0.5207097 (0)	total: 20.4ms	remaining: 6m 47s
500:	learn: 0.8694718	test: 0.8598465	best: 0.8598465 (500)	total: 7.44s	remaining: 4m 49s
1000:	learn: 0.8972045	test: 0.8853753	best: 0.8853753 (1000)	total: 14.8s	remaining: 4m 40s
1500:	learn: 0.9100721	test: 0.8954677	best: 0.8954677 (1500)	total: 22.1s	remaining: 4m 31s
2000:	learn: 0.9183486	test: 0.9010975	best: 0.9010975 (2000)	total: 29.4s	remaining: 4m 24s
2500:	learn: 0.9244371	test: 0.9044310	best: 0.9044310 (2500)	total: 36.8s	remaining: 4m 17s
3000:	learn: 0.9293296	test: 0.9068278	best: 0.9068278 (3000)	total: 44.2s	remaining: 4m 10s
3500:	learn: 0.9334166	test: 0.9084576	best: 0.9084576 (3500)	total: 51.5s	remaining: 4m 2s
4000:	learn: 0.9370300	test: 0.9096072	best: 0.9096072 (4000)	total: 59s	remaining: 3m 55s
4500:	learn: 0.9402561	test: 0.9105847	best: 0.9105847 (4500)	total: 1m 6s	remaining: 3m 48s
5000:	learn: 0.9431765	test: 0.9114830	best: 0.9114830 (5000)	total: 1m 13s	rema

0:	learn: 0.5161567	test: 0.5001065	best: 0.5001065 (0)	total: 21.8ms	remaining: 7m 15s
500:	learn: 0.8690305	test: 0.8675777	best: 0.8675777 (500)	total: 7.4s	remaining: 4m 48s
1000:	learn: 0.8967905	test: 0.8901767	best: 0.8901937 (999)	total: 14.8s	remaining: 4m 41s
1500:	learn: 0.9096988	test: 0.8997466	best: 0.8997466 (1500)	total: 22.3s	remaining: 4m 34s
2000:	learn: 0.9178003	test: 0.9050610	best: 0.9050610 (2000)	total: 29.7s	remaining: 4m 27s
2500:	learn: 0.9239839	test: 0.9084818	best: 0.9084870 (2498)	total: 37.1s	remaining: 4m 19s
3000:	learn: 0.9288974	test: 0.9105209	best: 0.9105209 (3000)	total: 44.4s	remaining: 4m 11s
3500:	learn: 0.9330331	test: 0.9119807	best: 0.9119813 (3499)	total: 51.6s	remaining: 4m 3s
4000:	learn: 0.9365779	test: 0.9132845	best: 0.9132845 (4000)	total: 58.9s	remaining: 3m 55s
4500:	learn: 0.9399040	test: 0.9143318	best: 0.9143318 (4500)	total: 1m 6s	remaining: 3m 47s
5000:	learn: 0.9428328	test: 0.9148826	best: 0.9148826 (5000)	total: 1m 13s	rema

In [17]:
pd.options.display.max_rows = 200

In [18]:
pd.Series(np.array(gains).sum(0)/10,index=train.columns).sort_values(ascending=False).head(200)

var_81                                0.772509
var_12                                0.535132
var_110                               0.528600
var_53                                0.483014
var_6                                 0.478856
var_146                               0.463165
var_139_train_test_sum_vcs_product    0.440544
var_22                                0.427525
var_2                                 0.417464
var_174                               0.416295
var_109                               0.409943
var_165                               0.403708
var_26_train_test_sum_vcs_product     0.403275
var_198                               0.397236
var_0                                 0.383023
var_76_train_test_sum_vcs_product     0.376113
var_166                               0.353064
var_78                                0.347913
var_133                               0.337524
var_179                               0.334243
var_34                                0.328852
var_94       

In [19]:
t = pd.DataFrame(clf.get_feature_importance(trn_data,'Interaction'))
t[0] = train.columns.values[t[0]]
t[1] = train.columns.values[t[1]]


In [20]:
t.sort_values(2,ascending=False).head(100)

,0,1,2
0,var_119,var_119_train_test_sum_vcs_product,0.043252
1,var_18,var_18_train_test_sum_vcs_product,0.040226
2,var_49,var_49_train_test_sum_vcs_product,0.037068
3,var_109,var_109_train_test_sum_vcs_div,0.034594
4,var_33,var_33_train_test_sum_vcs_div,0.033092
5,var_198,var_198_train_test_sum_vcs_product,0.031068
6,var_56,var_56_train_test_sum_vcs_div,0.030422
7,var_2,var_2_train_test_sum_vcs_div,0.027935
8,var_190,var_190_train_test_sum_vcs_product,0.027330
9,var_51,var_51_train_test_sum_vcs_product,0.025793


In [21]:
predictions

array([0.09536378, 0.19754081, 0.24315434, ..., 0.00247496, 0.06419023,
       0.02446914])

In [22]:
tst_sub = pd.read_csv('sample_submission.csv')

In [23]:
tst_sub['target'] = predictions

In [24]:
tst_sub.to_csv('_sub18.csv',index=False)

In [ ]:
pd.options.display.max_rows = 1000
pd.Series(model.feature_importances_,index=model.feature_names_).sort_values(ascending=False)